In [35]:
import sagemaker
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import boto3
import pandas as pd
import numpy as np

sm_boto3 = boto3.client("sagemaker")
sess = sagemaker.Session()
region = sess.boto_session.region_name
bucket = 'test-bucket-sagemaker3'
print("Using bucket " + bucket)

Using bucket test-bucket-sagemaker3


In [36]:
df = pd.read_csv('gabung0426_0330.csv', sep = ';')

In [37]:
df.head()

,label,acc_x_avg,acc_y_avg,acc_z_avg,gyro_x_avg,gyro_y_avg,gyro_z_avg,acc_x_max,acc_y_max,acc_z_max,...,acc_z_min,gyro_x_min,gyro_y_min,gyro_z_min,acc_x_stdev,acc_y_stdev,acc_z_stdev,gyro_x_stdev,gyro_y_stdev,gyro_z_stdev
0,2,1.91,-11.01,-0.45,5.28,22.35,-92.77,28.38,0.68,4.19,...,-5.56,-29.75,-249.99,-222.21,8.33,4.76,2.15,23.82,75.21,63.83
1,2,5.76,-9.60,-1.52,-25.25,-32.44,79.45,18.43,-1.40,4.99,...,-14.37,-107.72,-172.73,-330.50,6.73,6.13,4.45,44.53,89.72,211.71
2,2,1.88,-11.56,-0.38,7.05,29.11,-95.67,19.65,-1.18,9.83,...,-7.22,-27.63,-174.87,-245.47,7.91,5.22,3.05,24.00,59.02,65.90
3,2,6.73,-8.95,-0.53,-39.51,-30.96,94.43,14.97,-1.58,7.02,...,-16.78,-144.48,-178.11,-342.04,5.82,5.78,5.17,51.71,109.08,219.17
4,2,2.36,-11.03,-0.49,14.29,26.08,-99.75,20.98,0.50,10.68,...,-5.03,-23.37,-141.58,-239.12,9.46,5.55,3.18,32.55,74.98,79.86


In [38]:

# Pembagian dataframe label dan feature
features = list(df.columns)
label = features.pop(0)
y = df[label]
X = df[features]

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
X_train

,acc_x_avg,acc_y_avg,acc_z_avg,gyro_x_avg,gyro_y_avg,gyro_z_avg,acc_x_max,acc_y_max,acc_z_max,gyro_x_max,...,acc_z_min,gyro_x_min,gyro_y_min,gyro_z_min,acc_x_stdev,acc_y_stdev,acc_z_stdev,gyro_x_stdev,gyro_y_stdev,gyro_z_stdev
501,1.48,-11.78,-1.78,6.88,25.60,-129.71,40.25,24.72,3.79,68.54,...,-11.61,-57.21,-260.89,-244.20,14.28,10.05,3.48,29.75,92.93,74.44
436,6.15,-9.69,0.15,-22.85,-17.14,103.47,20.27,0.80,8.77,47.45,...,-10.36,-110.73,-185.30,-267.46,7.63,8.70,4.69,44.21,99.94,243.77
1331,7.30,-7.97,0.99,-31.78,-35.06,97.31,19.49,2.05,8.46,47.24,...,-11.17,-132.60,-194.66,-293.41,5.32,8.03,5.47,57.93,104.41,253.10
565,4.01,-10.95,-1.36,-20.33,-31.70,127.70,17.41,0.97,5.39,40.43,...,-11.89,-72.71,-186.91,-227.68,7.20,6.92,4.42,28.28,96.50,252.14
590,-0.33,-11.96,-1.12,4.81,29.70,-119.09,25.62,3.26,10.41,48.76,...,-10.24,-41.05,-135.32,-220.39,10.38,5.81,3.30,23.83,64.81,56.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
763,-0.41,-12.84,-0.65,4.95,34.03,-125.62,20.88,-0.43,12.49,67.48,...,-7.51,-50.06,-141.82,-224.71,8.56,5.48,3.34,29.78,76.57,53.62
835,-1.83,-13.24,-1.00,17.57,20.67,-129.50,18.64,-3.26,13.86,84.22,...,-7.66,-59.84,-135.90,-262.40,9.50,5.36,4.69,31.82,60.89,61.99
1216,1.27,-12.24,-0.92,19.71,25.08,-128.42,32.85,1.42,5.72,51.30,...,-4.79,-23.19,-253.95,-260.98,13.25,6.35,2.95,22.08,85.92,88.78
559,4.16,-10.34,-1.25,-22.82,-23.14,99.10,15.59,-0.50,4.64,25.11,...,-11.00,-66.09,-234.80,-235.13,6.26,5.97,3.69,28.36,105.02,234.32


In [39]:
# Normalisasi Data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [40]:
testX = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)
trainX = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
testX[label] = y_test
trainX[label] = y_train
testX

,acc_x_avg,acc_y_avg,acc_z_avg,gyro_x_avg,gyro_y_avg,gyro_z_avg,acc_x_max,acc_y_max,acc_z_max,gyro_x_max,...,gyro_x_min,gyro_y_min,gyro_z_min,acc_x_stdev,acc_y_stdev,acc_z_stdev,gyro_x_stdev,gyro_y_stdev,gyro_z_stdev,label
867,-1.177646,-1.135628,-0.220870,1.228347,0.865396,-0.978860,-0.586808,-0.815311,1.366405,0.950903,...,0.621970,0.736554,0.314962,-0.235497,-1.281217,-0.784293,-0.766630,-1.386508,-1.148959,0
459,-0.382782,-0.558319,-1.100777,0.716357,0.960831,-1.009644,1.564251,2.000403,-1.192953,0.931288,...,0.520472,-0.377915,0.532555,0.897565,0.397692,-0.856807,-0.839031,0.397629,-0.934966,1
211,1.088702,1.126481,0.430021,-0.473200,-1.238253,1.008330,-0.374799,-0.098127,-0.189684,-0.514578,...,-0.315476,-0.302655,-1.262907,-0.473882,0.787097,0.279243,0.492506,0.341588,1.225448,2
1335,1.328475,2.499063,2.696085,-1.670834,-1.146080,0.939859,-1.262140,-0.505555,4.707221,0.023419,...,-0.981556,-0.380610,0.187116,-1.733494,-0.221525,2.696370,1.207065,1.025677,0.826164,2
494,1.275922,0.990990,1.924659,-1.340576,-0.725188,0.991516,-1.010826,-0.650666,1.885102,0.153435,...,-1.051336,0.567170,-2.466189,-0.927107,1.067979,1.874547,1.599758,-1.125627,1.034255,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
333,-0.349936,-0.405155,-0.305244,0.900781,0.266684,-0.938249,1.297453,1.848316,0.339250,0.229090,...,1.008696,-0.526126,-0.371824,0.644466,0.474297,-1.231462,-0.877592,-0.198050,-1.028132,1
1314,-0.533872,-0.593664,-0.353458,1.203667,1.003247,-1.005826,1.193831,0.808817,-0.428558,0.471749,...,1.073072,0.164306,0.171008,1.180095,0.276402,-0.603008,-0.616321,-0.387430,-0.774674,2
466,1.006588,0.849609,0.900109,-0.895894,-1.442174,0.917766,-0.958419,-0.703687,0.991033,-1.861252,...,-0.850925,1.664124,-1.218928,-0.679893,0.474297,0.980210,0.505884,-1.987985,1.042819,1
795,-1.217061,-0.976573,-0.715064,0.772447,1.695762,-0.948321,-0.088944,-1.427848,-0.322771,1.106137,...,1.013865,1.874122,1.548671,-0.117776,-2.104713,-1.013920,-0.214185,-1.363802,-1.161227,0


In [41]:
trainX.to_csv("train-V-1.csv", index = False)
testX.to_csv("test-V-1.csv", index = False)

In [42]:
sk_prefix = "sagemaker/tugasakhir/sklearncontainer"
trainpath = sess.upload_data(
    path="train-V-1.csv", bucket=bucket, key_prefix=sk_prefix
)
testpath = sess.upload_data(
    path="test-V-1.csv", bucket=bucket, key_prefix=sk_prefix
)
trainpath

's3://test-bucket-sagemaker3/sagemaker/tugasakhir/sklearncontainer/train-V-1.csv'

In [43]:
%%writefile script.py

from sklearn import svm
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score
import sklearn
import joblib
import boto3
import pathlib
from io import StringIO
import argparse
import joblib
import os
import numpy as np
import pandas as pd

def model_fn(model_dir):
  clf = joblib.load(os.path.join(model_dir, "model.joblib"))
  return clf

if __name__ == "__main__":

  print("[INFO] Extracting arguments")
  parser = argparse.ArgumentParser()

  parser.add_argument("--kernel", type=str, default="rbf")
  parser.add_argument("--degree", type=int, default=3)
  parser.add_argument("--coef0", type=float, default=0.0)

  parser.add_argument("--model-dir", type=str, default=os.environ.get("SM_MODEL_DIR"))
  parser.add_argument("--train", type=str, default=os.environ.get("SM_CHANNEL_TRAIN"))
  parser.add_argument("--test", type=str, default=os.environ.get("SM_CHANNEL_TEST"))
  parser.add_argument("--train-file", type=str, default="train-V-1.csv")
  parser.add_argument("--test-file", type=str, default="test-V-1.csv")

  args, _ = parser.parse_known_args()

  print("SKLearn version: ", sklearn.__version__)
  print("Joblib version: ", joblib.__version__)
 
  print()
  train_df = pd.read_csv(os.path.join(args.train, args.train_file))
  test_df = pd.read_csv(os.path.join(args.test, args.test_file))

  features = list(train_df.columns)
  label = features.pop(-1)

  print("Building training and testing datasets")
  print()
  X_train = train_df[features]
  X_test = test_df[features]
  y_train = train_df[label]
  y_test = test_df[label]

  print("Training SVM model")
  print()
  model = svm.SVC(kernel=args.kernel, degree=args.degree, coef0=args.coef0) 
  model.fit(X_train, y_train)
  print()

  model_path = os.path.join(args.model_dir, "model.joblib")
  joblib.dump(model, model_path)
  print("Model path: " + model_path)
  print()

  y_pred_test = model.predict(X_test)
  test_acc = accuracy_score(y_test, y_pred_test)
  test_rep = classification_report(y_test, y_pred_test)

  print()
  print("Testing accuracy: ", test_acc)
  print("Testing report: ")
  print(test_rep)

Overwriting script.py


In [44]:
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION = "0.23-1"

sklearn_estimator = SKLearn(
    entry_point="script.py",
    role="arn:aws:iam::960558295345:role/service-role/AmazonSageMaker-ExecutionRole-20240424T165273",
    instance_count=1,
    instance_type="ml.m5.xlarge",
    framework_version=FRAMEWORK_VERSION,
    base_job_name="TestTA",
    hyperparameters={
        "kernel": "poly",
        "degree": 3,
        "coef0": 2,
    },
    use_spot_instances = True,
    max_wait = 7200,
    max_run = 3600
)

In [45]:
sklearn_estimator.fit({"train": trainpath, "test": testpath}, wait=True)

INFO:sagemaker:Creating training-job with name: TestTA-2024-05-02-01-32-14-434


2024-05-02 01:32:35 Starting - Starting the training job...
2024-05-02 01:32:54 Starting - Preparing the instances for training...
2024-05-02 01:33:30 Downloading - Downloading the training image..2024-05-02 01:34:03,671 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2024-05-02 01:34:03,674 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-05-02 01:34:03,710 sagemaker_sklearn_container.training INFO     Invoking user training script.
2024-05-02 01:34:03,857 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-05-02 01:34:03,869 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-05-02 01:34:03,880 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-05-02 01:34:03,890 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channel_input_dirs"

In [46]:
sklearn_estimator.latest_training_job.wait(logs="None")
artifact = sm_boto3.describe_training_job(
    TrainingJobName=sklearn_estimator.latest_training_job.name
)["ModelArtifacts"]["S3ModelArtifacts"]

print("Model artifact persisted at " + artifact)


2024-05-02 01:34:21 Starting - Preparing the instances for training
2024-05-02 01:34:21 Downloading - Downloading the training image
2024-05-02 01:34:21 Training - Training image download completed. Training in progress.
2024-05-02 01:34:21 Uploading - Uploading generated training model
2024-05-02 01:34:21 Completed - Training job completed
Model artifact persisted at s3://sagemaker-ap-southeast-2-960558295345/TestTA-2024-05-02-01-32-14-434/output/model.tar.gz


In [50]:
from sagemaker.sklearn.model import SKLearnModel
import time
from time import gmtime, strftime

model_name = "Custom-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model = SKLearnModel(
    name=model_name,
    model_data=artifact, #change this to model artifact location in S3
    role="arn:aws:iam::960558295345:role/service-role/AmazonSageMaker-ExecutionRole-20240424T165273",
    entry_point="script.py",
    framework_version=FRAMEWORK_VERSION,
)

In [51]:
model

In [52]:
from sagemaker.model_monitor import DataCaptureConfig, DatasetFormat, DefaultModelMonitor

data_capture_prefix = '{}/datacapture'.format(sk_prefix)
endpoint_name = "Custom-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("EndpointName={}".format(endpoint_name))

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m4.xlarge",
    endpoint_name=endpoint_name,
    data_capture_config=DataCaptureConfig(enable_capture=True,
                                          sampling_percentage=100,
                                          destination_s3_uri=f's3://{bucket}/{data_capture_prefix}'
                                          )
)


EndpointName=Custom-model-2024-05-02-01-48-35


INFO:sagemaker:Creating model with name: Custom-model-2024-05-02-01-48-32
INFO:sagemaker:Creating endpoint-config with name Custom-model-2024-05-02-01-48-35
INFO:sagemaker:Creating endpoint with name Custom-model-2024-05-02-01-48-35


-----!

In [53]:
print(predictor.predict(testX[features][0:1].values.tolist()))

[0]


In [54]:
count = 0
while count < 16:
    print((testX[label][(count):(count+2)]).values.tolist())
    count += 3


[0, 1]
[2, 1]
[0, 0]
[2, 1]
[0, 2]
[1, 0]


In [223]:
sm_boto3.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': '267b2342-704a-470b-b946-314388199549',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '267b2342-704a-470b-b946-314388199549',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Sat, 30 Mar 2024 17:52:11 GMT',
   'content-length': '0'},
  'RetryAttempts': 0}}